In [ ]:
# gdown 설치 및 import
!pip install -q gdown
import gdown

In [ ]:
# 구글 드라이브에서 파일 다운로드
file_id = '19S1vMH8LrZQV-waa0VjYUY75ZhO2pLSx'
gdown.download(f"https://drive.google.com/uc?id={file_id}", "bike_data.csv", quiet=False)

In [ ]:
# CSV 파일 불러오기 (cp949 인코딩)
import pandas as pd
df = pd.read_csv("bike_data.csv", encoding='cp949')

In [ ]:
# 열 이름 영어로 변경
df = df.rename(columns={
    '기준_날짜': 'RENT_DATE',
    '기준_시간대': 'RENT_TIME',
    '시작_대여소명': 'RENT_STATION_NAME',
    '종료_대여소명': 'RETURN_STATION_NAME',
    '전체_건수': 'RENT_COUNT',
    '전체_이용_분': 'RENT_DURATION_MIN',
    '전체_이용_거리': 'RENT_DISTANCE_METER'
})

In [ ]:
# 시간 파싱 및 시간대 그룹화
df['RENT_DATE'] = pd.to_datetime(df['RENT_DATE'], format='%Y%m%d')
df['hour'] = df['RENT_TIME'].astype(str).str.zfill(4).str[:2].astype(int)

def group_time(hour):
    if 6 <= hour < 9:
        return '06~09'
    elif 9 <= hour < 12:
        return '09~12'
    elif 12 <= hour < 15:
        return '12~15'
    elif 15 <= hour < 18:
        return '15~18'
    elif 18 <= hour < 21:
        return '18~21'
    else:
        return '기타'

df['time_slot'] = df['hour'].apply(group_time)

In [ ]:
# 피벗 테이블 생성
pivot_table = pd.pivot_table(
    df,
    values='RENT_COUNT',
    index='RENT_STATION_NAME',
    columns='time_slot',
    aggfunc='sum',
    fill_value=0
)

print("모든 대여소의 시간대별 대여 건수 요약표:")
print(pivot_table)

In [ ]:
# 저장 및 다운로드
output_file = "rent_summary_by_station_timeslot_cp949.csv"
pivot_table.to_csv(output_file, encoding='cp949')

from google.colab import files
files.download(output_file)